进程是操作系统分配资源的最小单位；线程是CPU调度的最小单位；  
多进程之间的资源是独立的，多线程之间的资源是共享的；  
一个进程里可以有多个线程。  

### 多进程
python提供两种方法支持多进程：os模块的fork方法和multiprocessing模块，后者还提供进程通信机制。

In [ ]:
# (1) os模块中封装了Unix/Linux系统调用fork()
import os

print('Process (%s) start...' % os.getpid())
# Only works on Unix/Linux/Mac:
pid = os.fork()
if pid == 0:
    print('I am child process (%s) and my parent is %s.' % (os.getpid(), os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))

# 上面这段代码只能在Linux上运行，无法在windows上运行

In [ ]:
# (2) multiprocessing是一个跨平台版本的多进程模块
from multiprocessing import Process
import os

# 可以用multiprocessing.Process模块创建子进程
# 定义子进程要调用的函数
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

print('Parent process %s.' % os.getpid())
p = Process(target=run_proc, args=('test',))
print('Child process will start.')
p.start()
p.join()
print('Child process end.')

# 也可以用可以用multiprocessing.Pool批量创建子进程
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

In [ ]:
# (3) 有时候子进程并不是我们主进程创建的，而是外部进程。subprocess模块即可实现启动一个外部进程作为子进程
import subprocess

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.python.org'])
print('Exit code:', r)

# 这段代码相当于在命令行中运行命令：nslookup www.python.org

In [ ]:
# (4) 进程间通信：
# multiprocessing模块封装了操作系统的进程通信机制，提供了Queue、Pipes等多种方式来交换数据。

### 多线程
一个进程可能包含多个线程，多个线程共享该进程中的全部系统资源，如虚拟地址空间、文件描述符、信号处理等。任何一个进程都至少有一个线程，称为主线程。  
python的线程是真正的Posix Thread。  

In [ ]:
# (1) python提供threading模块实现线程的创建和操作
import time, threading

# 新线程执行的代码:
def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t = threading.Thread(target=loop, name='LoopThread')
t.start()
t.join()
print('thread %s ended.' % threading.current_thread().name)

In [ ]:
# (2) 线程锁：threading.Lock()
balance = 0
lock = threading.Lock()

def run_thread(n):
    for i in range(100000):
        # 先要获取锁:
        lock.acquire()
        try:
            # 放心地改吧:
            change_it(n)
        finally:
            # 改完了一定要释放锁:
            lock.release()

In [ ]:
# (3) GIL锁（Global Interpreter Lock）：
# python解释器执行代码时，有一个GIL锁，任何Python线程执行前，必须先获得GIL锁，然后，每执行100条字节码，解释器就自动释放GIL锁，
# 让别的线程有机会执行。这个GIL全局锁实际上把所有线程的执行代码都给上了锁，
# 所以，多线程在Python中只能交替执行，即使100个线程跑在100核CPU上，也只能用到1个核。
# GIL锁能保证同一时刻只有一个线程在运行。
# GIL锁只对python代码编译成的pyc字节码有效（因为毕竟它是python解释器的机制嘛），对于调用的外部函数（如C、C++扩展程序），不会使用GIL锁。

### 协程
协程是单线程的异步程序。  
协程与一般的调用函数不同，一般的函数就是顺序执行，而协程在执行过程中，可以内部中断，转而执行别的子程序，在适当的时候再返回来接着执行（注意，在程序中中断，去执行其他子程序这一过程，不是函数调用，而是有点类似于CPU的中断）。  
协程与多线程不同，协程比多线程运行效率高，因为协程的切换不是系统控制，而是由程序自身控制，没有线程切换的开销。  
Python对协程的支持是通过生成器（generator）实现的。  

### 面试题

In [2]:
# (1) 简述GIL:
# Python代码的执行由Python 虚拟机(也叫解释器主循环，CPython版本)来控制，Python 在设计之初就考虑到要在解释器的主循环中，同时只有一个线程在执行，即在任意时刻，只有一个线程在解释器中运行。对Python 虚拟机的访问由全局解释器锁（GIL）来控制，正是这个锁能保证同一时刻只有一个线程在运行。

# 在多线程环境中，Python 虚拟机按以下方式执行：
# （1）. 设置GIL
# （2）. 切换到一个线程去运行
# （3）. 运行：
#     a. 指定数量的字节码指令，或者
#     b. 线程主动让出控制（可以调用time.sleep(0)）
# （4）. 把线程设置为睡眠状态
# （5）. 解锁GIL
# （6）. 再次重复以上所有步骤
#  在调用外部代码（如C/C++扩展函数）的时候，GIL 将会被锁定，直到这个函数结束为止（由于在这期间没有Python 的字节码被运行，所以不会做线程切换）。

In [3]:
# (2) 简述“乐观锁”和“悲观锁”

In [ ]:
# (3) 简述协程